In [49]:
import pandas as pd
from tqdm import tqdm
import ast #eval() to interpret code
import warnings
warnings.filterwarnings('ignore')
tqdm.pandas()
from datetime import datetime, timezone
import matplotlib.pyplot as plt
import json
import numpy as np
from web3 import Web3
import time
from datetime import datetime, timedelta
from scipy.stats import norm
from scipy.special import kl_div
from datetime import datetime, timedelta

In [50]:
def calc_debt_time(df_debt, df_rate):
    '''
        Calculate debt for each user at each timestamp

        return:
                a column which shows how a user's debt changes over time
    '''

    debt_arr = []
    isFirstDebt = True

    for i in range(df_debt.shape[0]):

        row = df_debt.iloc[i]
        action = row.user_action

        if (action == 'lock asset') | (action == 'unlock asset'):
            continue

        if isFirstDebt & (action == 'transfer in'):
            isFirstDebt = False
            debt_arr.append(row.dai)
            prev_date = pd.to_datetime(row.timestamp)
            prev_date_sec = int(datetime.fromisoformat(row.timestamp)                         # in seconds
                                    .replace(tzinfo=timezone.utc)
                                    .timestamp())
            continue

        
        if isFirstDebt & (action == 'generate dai'):
            isFirstDebt = False
            debt_arr.append(row.dai)
            prev_date = pd.to_datetime(row.timestamp)
            prev_date_sec = int(datetime.fromisoformat(row.timestamp)                         # in seconds
                                    .replace(tzinfo=timezone.utc)
                                    .timestamp())
            continue
            
        else:
            new_date = pd.to_datetime(row.timestamp)
            new_date_sec = int(datetime.fromisoformat(row.timestamp)                         # in seconds
                                        .replace(tzinfo=timezone.utc)
                                        .timestamp())
            
            closest_index_prev_date = np.abs(pd.to_datetime(df_rate['timestamp']) - prev_date).idxmin()
            closest_index_new_date = np.abs(pd.to_datetime(df_rate['timestamp']) - new_date).idxmin()
            rate_prev_date = df_rate['percentage'][closest_index_prev_date]
            rate_new_date = df_rate['percentage'][closest_index_new_date]
            rate_prev_date_timestamp = df_rate['timestamp'][closest_index_prev_date]         # timestamp in the rate table
            rate_new_date_timestamp = df_rate['timestamp'][closest_index_new_date]           # timestamp in the rate table

            if rate_prev_date_timestamp == rate_new_date_timestamp:                          # f(t) is constant during (prev_date; now_date]
                delta_t = new_date_sec - prev_date_sec - 1                                   # in seconds
                if debt_arr[-1] == 0:
                    if row.dai > 0:
                        debt_arr.append(row.dai)
                    else:
                        debt_arr.append(0)
                    rate_prev_date = rate_new_date
                    continue
                else:
                    debt = debt_arr[-1] * np.exp(rate_prev_date / 100 * (delta_t / (365.25 * 24 * 60 * 60)))

            else:
                cond1 = pd.to_datetime(df_rate['timestamp']) >= pd.to_datetime(rate_prev_date_timestamp)
                cond2 = pd.to_datetime(df_rate['timestamp']) <= pd.to_datetime(rate_new_date_timestamp)
                df_rate_cond = df_rate[cond1 & cond2]

                for j in range(df_rate_cond.shape[0]):
                    row1 = df_rate_cond.iloc[j]
                    if j == 0:
                        timestamp_prev = int(datetime.fromisoformat(row1.timestamp)
                                                    .replace(tzinfo=timezone.utc)
                                                    .timestamp())
                        rate_prev = row1.percentage
                        debt = debt_arr[-1]

                    else:
                        timestamp_new = int(datetime.fromisoformat(row1.timestamp)
                                                    .replace(tzinfo=timezone.utc)
                                                    .timestamp())
                        rate_new = row1.percentage
                        delta_t = timestamp_new - timestamp_prev - 1
                        if debt_arr[-1] == 0:
                            if row.dai > 0:
                                debt_arr.append(row.dai)
                            else:
                                debt_arr.append(0)

                            rate_prev_date = rate_new_date
                            continue
                        else:
                            debt = debt * np.exp(rate_prev / 100 * (delta_t / (365.25 * 24 * 60 * 60)))

                        rate_prev = rate_new
                        timestamp_prev = timestamp_new

        rate_prev_date = rate_new_date

        #print('THE FINAL DEBT: ', debt + row.dai)
        #print('THE ROW.DAI: ', row.dai)
        debt_final = debt + row.dai
        if debt_final < 0:
            debt_arr.append(0)
        else:
            debt_arr.append(debt_final)
        prev_date = new_date
        prev_date_sec = new_date_sec
    
    return debt_arr

In [51]:
def add_debt_column(df, debt_arr):
    '''
        Add a debt column to the table
        
        return:
                a table with the debt column
    '''
    new_col = None
    df.insert(loc = 8, column = 'debt', value = new_col)

    j = 0
    isFirstIter = True

    for i in range(df.shape[0]):
        row = df.iloc[i]

        if (row.user_action == 'lock asset') & isFirstIter:
            df['debt'].iloc[i] = 0
            isFirstIter = False
            continue

        if (row.user_action == 'lock asset') | (row.user_action == 'unlock asset'):
            df['debt'].iloc[i] = df['debt'].iloc[i-1]
        else:
            df['debt'].iloc[i] = debt_arr[j]
            j += 1
    return df

In [52]:
def add_f_column(df, df_rate):
    '''
        Add a annual interest rate (f) column

        return:
                a table with the f column     
    '''
    new_col = None
    df.insert(loc = 9, column = 'f', value = new_col)
    for i in range(df.shape[0]):
        row = df.iloc[i]
        closest_index = np.abs(pd.to_datetime(df_rate['timestamp']) - pd.to_datetime(row.timestamp)).idxmin()
        f = df_rate['percentage'][closest_index]
        df['f'].iloc[i] = f
    
    return df

In [53]:
def psi_t(y, drift, t, valot):
    '''
        Calculate probability of default of one user in one asset
    '''
    return np.exp(2*y*drift / (valot**2)) * (1 - norm.cdf((drift*t + y)/(valot*np.sqrt(t)))) + (1 - norm.cdf((y - drift*t)/(valot*np.sqrt(t))))

In [54]:
def find_closest_rate(timestamp, df_dai_rate):
  '''
    Find the closest exchange rate for a given date

    return:
            the most suitable rate 
  '''

  closest_index = np.abs(pd.to_datetime(df_dai_rate['timestamp']) - timestamp).idxmin()
  time_rate = pd.to_datetime(df_dai_rate['timestamp'][closest_index])
  time_data = timestamp

  if time_data == time_rate:
    rate = df_dai_rate['rate'][closest_index]

  elif time_data < time_rate:

    if closest_index == 0:
      rate = df_dai_rate['rate'][0]
    else:
      rate = df_dai_rate['rate'][closest_index-1]

  else:

    if closest_index == df_dai_rate.shape[0]-1:
      rate = df_dai_rate['rate'][df_dai_rate.shape[0]-1]
    else:
      rate = df_dai_rate['rate'][closest_index]

  return rate

In [55]:
def sigma_calculation(df, df_usd_rate):
    '''
        Calculate a constant for Brownian motion
    '''
    df = df.sort_values(by=['timestamp'])
    first_date = datetime.strptime(df.iloc[0].timestamp, '%Y-%m-%d %H:%M:%S%z')
    last_date = datetime.strptime(df.iloc[-1].timestamp, '%Y-%m-%d %H:%M:%S%z')

    current_date = first_date
    prev_e = find_closest_rate(pd.to_datetime(current_date), df_usd_rate)

    sum_e_log = 0
    days = 0

    while current_date <= last_date:        
        current_date += timedelta(days=1)
        days += 1
        cur_e = find_closest_rate(pd.to_datetime(current_date), df_usd_rate)
        sum_e_log += (np.log(cur_e / prev_e))**2
        prev_e = cur_e
    
    return np.sqrt(sum_e_log / days)

In [56]:
def sigma_for_ilk(df, df_usd_rate):
    '''
        Add a columt with the sigma value to the table

        return:
                a table with the sigma column
    '''
    df = df.sort_values(by=['timestamp'])
    new_col = None
    df.insert(loc = 8, column = 'sigma', value = new_col)
    df['sigma'] = sigma_calculation(df, df_usd_rate)
    return df

In [57]:
def psi_x_min(df, isWBTC):
    '''
        Calculate theoretical probability of default
        
        returtn:
                an array containing theoretical probabilities of default the next day for each user at each timestamp
                a real flag of if user has defaulted or not
                timestamps array
                exchange rate array
                passage level array
                collateral ratio array
    '''
    
    psi_arr = []
    isDefault = []

    t_arr = []
    e0_arr = []
    x_min_arr = []
    collateral_ratio = []

    a0 = 0
    t = 1

    for i in range(df.shape[0]):
        row = df.iloc[i]
        action = row.user_action

        if (action == 'lock asset') | (action == 'unlock asset'):
            a0 += row.lock_collateral
            x_min_arr.append(None)
            continue

        t_tmp = ((datetime.fromisoformat(row.timestamp) + timedelta(days=1)).isoformat())
        t_arr.append(t_tmp.split('T')[0])

        if isWBTC:
            e0 = row.WBTC_price             # WBTC/DAI
        else:
            e0 = row.ETH_price              # ETH/DAI
        d0 = row.debt                       # DAI
        a0 += row.lock_collateral           # ETH
        r_min = row.Liquidate_Rate
        f = row.f  / (100*365)
        sigma = row.sigma

        if (a0 <= 0):
            a0 = 0
            x_min_arr.append(None)
            continue

        if (d0 == 0):
            x_min_arr.append(None)
            continue

        x_min = np.log((d0 * r_min) / (a0 * e0)) / sigma

        e0_arr.append(e0)
        x_min_arr.append(-x_min)
        # print(x_min)
        if np.isnan(x_min) | np.isinf(x_min):
            print('x_min', x_min)
            print('e0', e0)
            print('d0', d0)
            print('a0', a0)
            print('r_min', r_min)

        psi = psi_t(np.abs(x_min), -f, t, sigma)

        psi_arr.append(psi)
        collateral_ratio.append(row['CR_Rate'])

        d_t = d0 * np.exp(f*t)
        r_t = e0 * a0 / d_t

        if r_t >= r_min:
            isDefault.append(0)
        else:
            isDefault.append(1)
    
    return psi_arr, isDefault, t_arr, e0_arr, x_min_arr, collateral_ratio
    

In [58]:
def debt_each_user(df, df_rate):
    '''
        Separate users from each other within the same table and add new parameters for them

        return:
                a table with new columns such as debt and annual unterest rate
    '''

    df['dai'] = df.apply(lambda row: row['dai(in exchange)'] if np.isnan(row['dai']) else row['dai'], axis=1)
    users = df.usr.unique()
    
    for user in users:

        # if user == '0x01f1C7d9d3B8518c84880a5240131dC760F93ECc':
        #     continue

        df_user = df[df['usr'] == user]
        df_user = df_user.sort_values(by=['timestamp'])
        debt_at_time = calc_debt_time(df_user, df_rate)

        df_debt = add_debt_column(df_user, debt_at_time)
        df_cur_user = add_f_column(df_debt, df_rate)

        if user == users[0]:
            df_new = df_cur_user
        else:
            df_new = pd.concat([df_new, df_cur_user], ignore_index=True, sort=False)
    
    return df_new

In [59]:
def default_prob_each_user(df, isWBTC):
    '''
        Separate users from each other within the same table and calculate theoretiacal and real probabilities of default

        return:
                theoretical probability of default
                real probability of default
                passage level array
    '''

    theor_prob = []
    real_prob = []
    x_min_arr = []

    users = df.usr.unique()
    for user in users:
        if user == '0x01f1C7d9d3B8518c84880a5240131dC760F93ECc':
            continue
        
        df_user = df[df['usr'] == user]
        theor_prob_tmp, real_prob_tmp, _, _, x_min_arr_tmp, _ = psi_x_min(df_user, isWBTC)

        theor_prob += theor_prob_tmp
        real_prob += real_prob_tmp
        x_min_arr += x_min_arr_tmp
            
    return theor_prob, real_prob, x_min_arr

In [71]:
data_all_ilk_all_usr = pd.read_csv('data_all_ilk_all_usr.csv')
rate_7_ilk = pd.read_csv('rate_makerdao_top7_progs.csv')
# df_eth_dai = pd.read_csv('USD_ETH.csv')
df_eth_usd = pd.read_csv('USD_ETH.csv')
df_eth_usd.rate[0] = 181.439693

wbtc_usd_rate = pd.read_csv('wbtc-usd-max.csv')
wbtc_usd_rate = wbtc_usd_rate.rename(columns={'snapped_at': 'timestamp', 'price': 'rate'})

In [43]:
debt_status_eth_a = pd.read_csv('debt_status_eth_a.csv')
debt_status_gun_a = pd.read_csv('debt_status_gun_a.csv')
debt_status_wbtc_a = pd.read_csv('debt_status_wbtc_a.csv')
debt_status_eth_c = pd.read_csv('debt_status_eth_c.csv')
debt_status_eth_b = pd.read_csv('debt_status_eth_b.csv')

In [ ]:
df_eth_a = debt_each_user(data_all_ilk_all_usr[data_all_ilk_all_usr['ilk'] == 'ETH-A'], rate_7_ilk[rate_7_ilk['ilk'] == 'ETH-A'])
df_eth_b = debt_each_user(data_all_ilk_all_usr[data_all_ilk_all_usr['ilk'] == 'ETH-B'], rate_7_ilk[rate_7_ilk['ilk'] == 'ETH-B'])
df_eth_c = debt_each_user(data_all_ilk_all_usr[data_all_ilk_all_usr['ilk'] == 'ETH-C'], rate_7_ilk[rate_7_ilk['ilk'] == 'ETH-C'])
df_gun_a = debt_each_user(data_all_ilk_all_usr[data_all_ilk_all_usr['ilk'] == 'GUNIV3DAIUSDC2-A'], rate_7_ilk[rate_7_ilk['ilk'] == 'GUNIV3DAIUSDC2-A'])
df_wbtc_a = debt_each_user(data_all_ilk_all_usr[data_all_ilk_all_usr['ilk'] == 'WBTC-A'], rate_7_ilk[rate_7_ilk['ilk'] == 'WBTC-A'])

In [ ]:
def add_wbtc_dai_rate(df, df_wbtc_usd_rate=wbtc_usd_rate):
    new_col = None
    df.insert(loc = 8, column = 'WBTC_price', value = new_col)
    for i in range(df.shape[0]):
        row = df.iloc[i]
        dai_usd_rate = row['rate(DAI/USD)']
        wbtc_dai_rate = find_closest_rate(pd.to_datetime(row.timestamp), df_wbtc_usd_rate) / dai_usd_rate
        df['WBTC_price'][i] = wbtc_dai_rate
    return df

df_wbtc_a = add_wbtc_dai_rate(df_wbtc_a).copy()

In [ ]:
df_eth_a = df_eth_a.sort_values(by=['timestamp'])
df_eth_a.ETH_price[:7] = 184.970907

In [676]:
df_eth_a.to_csv('df_eth_a_debt_all_usrs.csv')
df_eth_b.to_csv('df_eth_b_debt_all_usrs.csv')
df_eth_c.to_csv('df_eth_c_debt_all_usrs.csv')
df_gun_a.to_csv('df_gun_a_debt_all_usrs.csv')
df_wbtc_a.to_csv('df_wbtc_debt_all_usrs.csv')

In [4]:
df_eth_a = pd.read_csv('df_eth_a_debt_all_usrs.csv')
df_eth_b = pd.read_csv('df_eth_b_debt_all_usrs.csv')
df_eth_c = pd.read_csv('df_eth_c_debt_all_usrs.csv')
df_gun_a = pd.read_csv('df_gun_a_debt_all_usrs.csv')
df_wbtc_a = pd.read_csv('df_wbtc_debt_all_usrs.csv')

In [76]:
df_wbtc_a = pd.read_csv('wbtc_a_debt_sigma_new.csv')
df_wbtc_a = df_wbtc_a.drop('sigma', axis=1)

In [77]:
sigma_eth_a_new = sigma_for_ilk(df_eth_a, df_eth_usd)
sigma_eth_b_new = sigma_for_ilk(df_eth_b, df_eth_usd)
sigma_eth_c_new = sigma_for_ilk(df_eth_c, df_eth_usd)
sigma_gun_a_new = sigma_for_ilk(df_gun_a, df_eth_usd)
sigma_wbtc_a_new = sigma_for_ilk(df_wbtc_a, wbtc_usd_rate)

In [222]:
sigma_eth_a_new.to_csv('eth_a_debt_sigma_new.csv')
sigma_gun_a_new.to_csv('gun_a_debt_sigma_new.csv')
sigma_eth_b_new.to_csv('eth_b_debt_sigma_new.csv')
sigma_eth_c_new.to_csv('eth_c_debt_sigma_new.csv')
sigma_wbtc_a_new.to_csv('wbtc_a_debt_sigma_new.csv')

In [78]:
theor_eth_a, real_eth_a, x_min_arr_eth_a = default_prob_each_user(sigma_eth_a_new, False)
theor_eth_b, real_eth_b, x_min_arr_eth_b = default_prob_each_user(sigma_eth_b_new, False)
theor_eth_c, real_eth_c, x_min_arr_eth_c = default_prob_each_user(sigma_eth_c_new, False)
theor_gun_a, real_gun_a, x_min_arr_gun_a = default_prob_each_user(sigma_gun_a_new, False)
theor_wbtc_a, real_wbtc_a, x_min_arr_wbtc_a = default_prob_each_user(sigma_wbtc_a_new, True)

In [79]:
d = {'theor_prob': theor_eth_a, 'real': real_eth_a}
pd.DataFrame.from_dict(d).to_csv('prob_eth_a')

d = {'theor_prob': theor_eth_b, 'real': real_eth_b}
pd.DataFrame.from_dict(d).to_csv('prob_eth_b')

d = {'theor_prob': theor_eth_c, 'real': real_eth_c}
pd.DataFrame.from_dict(d).to_csv('prob_eth_c')

d = {'theor_prob': theor_gun_a, 'real': real_gun_a}
pd.DataFrame.from_dict(d).to_csv('prob_gun_a')

d = {'theor_prob': theor_wbtc_a, 'real': real_wbtc_a}
pd.DataFrame.from_dict(d).to_csv('prob_wbtc_a')

In [80]:
def mse(pred, real):
    mse = 0
    for i in range(len(pred)):
        if np.isnan(pred[i]) | np.isnan(real[i]):
            continue
        mse += (pred[i] - real[i])**2
    return mse / len(pred)

def mae(pred, real):
    mae = 0
    for i in range(len(pred)):
        if np.isnan(pred[i]) | np.isnan(real[i]):
            continue
        mae += np.abs(pred[i] - real[i])
    return mae / len(pred)

def kl(pred, real):
    kl = kl_div(pred, real)
    kl = kl[~np.isnan(kl) & ~np.isinf(kl)]
    return np.mean(kl)

def total_variation(pred, real):
    tv = 0
    for i in range(len(pred)):
        if np.isnan(pred[i]) | np.isnan(real[i]):
            continue
        tv += np.abs(pred[i] - real[i])
    return 0.5 * tv / len(pred)

In [83]:
'''
    Calculate metrics
'''

print('----------------- ETH-A -----------------')
print('MSE: ', mse(theor_eth_a, real_eth_a))
print('MAE: ', mae(theor_eth_a, real_eth_a))
print('KL: ', kl(theor_eth_a, real_eth_a))
print('TV: ', total_variation(theor_eth_a, real_eth_a))

print('----------------- ETH-B -----------------')
print('MSE: ', mse(theor_eth_b, real_eth_b))
print('MAE: ', mae(theor_eth_b, real_eth_b))
print('KL: ', kl(theor_eth_b, real_eth_b))
print('TV: ', total_variation(theor_eth_b, real_eth_b))

print('----------------- ETH-C -----------------')
print('MSE: ', mse(theor_eth_c, real_eth_c))
print('MAE: ', mae(theor_eth_c, real_eth_c))
print('KL: ', kl(theor_eth_c, real_eth_c))
print('TV: ', total_variation(theor_eth_c, real_eth_c))

print('----------------- GUNIV3DAIUSDC2-A -----------------')
print('MSE: ', mse(theor_gun_a, real_gun_a))
print('MAE: ', mae(theor_gun_a, real_gun_a))
print('KL: ', kl(theor_gun_a, real_gun_a))
print('TV: ', total_variation(theor_gun_a, real_gun_a))

print('----------------- WBTC-A -----------------')
print('MSE: ', mse(theor_wbtc_a, real_wbtc_a))
print('MAE: ', mae(theor_wbtc_a, real_wbtc_a))
print('KL: ', kl(theor_wbtc_a, real_wbtc_a))
print('TV: ', total_variation(theor_wbtc_a, real_wbtc_a))

----------------- ETH-A -----------------
MSE:  0.04730079990292176
MAE:  0.04857531788898582
KL:  0.045917361420327686
TV:  0.02428765894449291
----------------- ETH-B -----------------
MSE:  0.17328605179591508
MAE:  0.1807212811411858
KL:  0.17455827981443836
TV:  0.0903606405705929
----------------- ETH-C -----------------
MSE:  0.04993793400552888
MAE:  0.0504361591245414
KL:  0.049616208557872694
TV:  0.0252180795622707
----------------- GUNIV3DAIUSDC2-A -----------------
MSE:  0.003703703703703704
MAE:  0.003703703703703704
KL:  0.003703703703703704
TV:  0.001851851851851852
----------------- WBTC-A -----------------
MSE:  0.11515729528412087
MAE:  0.11600049269281541
KL:  0.11537147987716001
TV:  0.058000246346407706


In [782]:
new_col = None
sigma_eth_a_new.insert(loc = 10, column = 'x_min', value = new_col)
sigma_eth_a_new['x_min'] = x_min_arr_eth_a
sigma_eth_a_new.to_csv('df_eth_a_with_x_min.csv')

In [783]:
new_col = None
df_wbtc_a.insert(loc = 10, column = 'x_min', value = new_col)
df_wbtc_a['x_min'] = x_min_arr_wbtc_a
df_wbtc_a.to_csv('df_wbtc_a_with_x_min.csv')

## Poisson Model

In [5]:
real_eth_a = pd.read_csv('prob_eth_a')
real_eth_b = pd.read_csv('prob_eth_b')
real_eth_c = pd.read_csv('prob_eth_c')
real_gun_a = pd.read_csv('prob_gun_a')
real_wbtc_a = pd.read_csv('prob_wbtc_a')

In [3]:
def poisson_prob(df_status):

    N = 0
    M = 0
    X_n = 0
    Y_m = 0

    for i in range(df_status.shape[0]):
        row = df_status.iloc[i]
        start_date = int(datetime.fromisoformat(row.start_date)                         # in seconds
                                 .replace(tzinfo=timezone.utc)
                                 .timestamp())
        end_date = int(datetime.fromisoformat(row.end_date)                             # in seconds
                               .replace(tzinfo=timezone.utc)
                               .timestamp())
        delta = end_date - start_date

        if (row.status == 'liquidate') | (row.status == 'restruct'):
            N += delta
            for x in range(start_date, end_date):
                X_n += x
        else:
            M += delta
            for y in range(start_date, end_date):
                Y_m += y
    
    first_date = int(datetime.fromisoformat(df_status.sort_values(by=['start_date']).iloc[0].start_date)
                             .replace(tzinfo=timezone.utc)
                             .timestamp())
    
    last_date = int(datetime.fromisoformat(df_status.sort_values(by=['end_date']).iloc[-1].end_date)
                            .replace(tzinfo=timezone.utc)
                            .timestamp())
    
    T = last_date - first_date
    return 1 - np.exp(-((N + M) / (X_n + Y_m)) * T)

In [4]:
def mse_poisson(pred_prob, real):
    pred = [pred_prob] * len(real)
    mse = 0
    for i in range(len(pred)):
        if np.isnan(pred[i]) | np.isnan(real[i]):
            continue
        mse += (pred[i] - real[i])**2
    return mse / len(pred)

def mae_poisson(pred_prob, real):
    pred = [pred_prob] * len(real)
    mae = 0
    for i in range(len(pred)):
        if np.isnan(pred[i]) | np.isnan(real[i]):
            continue
        mae += np.abs(pred[i] - real[i])
    return mae / len(pred)

def kl_poisson(pred_prob, real):
    pred = [pred_prob] * len(real)
    kl = kl_div(pred, real)
    kl = kl[~np.isnan(kl) & ~np.isinf(kl)]
    return np.mean(kl)

def total_variation_poisson(pred_prob, real):
    pred = [pred_prob] * len(real)
    tv = 0
    for i in range(len(pred)):
        if np.isnan(pred[i]) | np.isnan(real[i]):
            continue
        tv += np.abs(pred[i] - real[i])
    return 0.5 * tv / len(pred)

In [728]:
theor_gun_a_poisson = poisson_prob(debt_status_gun_a)
theor_eth_a_poisson = poisson_prob(debt_status_eth_a)
theor_eth_b_poisson = poisson_prob(debt_status_eth_b)
theor_eth_c_poisson = poisson_prob(debt_status_eth_c)
theor_wbtc_a_poisson = poisson_prob(debt_status_wbtc_a)

In [729]:
print('----------------- ETH-A -----------------')
print('MSE: ', mse_poisson(theor_eth_a_poisson, real_eth_a.real))
print('MAE: ', mae_poisson(theor_eth_a_poisson, real_eth_a.real))
print('KL: ', kl_poisson(theor_eth_a_poisson, real_eth_a.real))
print('TV: ', total_variation_poisson(theor_eth_a_poisson, real_eth_a.real))

print('----------------- ETH-B -----------------')
print('MSE: ', mse_poisson(theor_eth_b_poisson, real_eth_b.real))
print('MAE: ', mae_poisson(theor_eth_b_poisson, real_eth_b.real))
print('KL: ', kl_poisson(theor_eth_b_poisson, real_eth_b.real))
print('TV: ', total_variation_poisson(theor_eth_b_poisson, real_eth_b.real))

print('----------------- ETH-C -----------------')
print('MSE: ', mse_poisson(theor_eth_c_poisson, real_eth_c.real))
print('MAE: ', mae_poisson(theor_eth_c_poisson, real_eth_c.real))
print('KL: ', kl_poisson(theor_eth_c_poisson, real_eth_c.real))
print('TV: ', total_variation_poisson(theor_eth_c_poisson, real_eth_c.real))

print('----------------- WBTC-A -----------------')
print('MSE: ', mse_poisson(theor_wbtc_a_poisson, real_wbtc_a.real))
print('MAE: ', mae_poisson(theor_wbtc_a_poisson, real_wbtc_a.real))
print('KL: ', kl_poisson(theor_wbtc_a_poisson, real_wbtc_a.real))
print('TV: ', total_variation_poisson(theor_wbtc_a_poisson, real_wbtc_a.real))

print('----------------- GUNIV3DAIUSDC2-A -----------------')
print('MSE: ', mse_poisson(theor_gun_a_poisson, real_gun_a.real))
print('MAE: ', mae_poisson(theor_gun_a_poisson, real_gun_a.real))
print('KL: ', kl_poisson(theor_gun_a_poisson, real_gun_a.real))
print('TV: ', total_variation_poisson(theor_gun_a_poisson, real_gun_a.real))

----------------- ETH-A -----------------
MSE:  0.04820386782117881
MAE:  0.1132799715137966
KL:  0.7439256918156401
TV:  0.0566399857568983
----------------- ETH-B -----------------
MSE:  0.16735926722757186
MAE:  0.21672447011295015
KL:  0.7940330892682707
TV:  0.10836223505647508
----------------- ETH-C -----------------
MSE:  0.04899774972716107
MAE:  0.09128278462131849
KL:  0.8178065407714942
TV:  0.045641392310659244
----------------- WBTC-A -----------------
MSE:  0.003960941181032167
MAE:  0.06115063147971626
KL:  0.7686747828984932
TV:  0.03057531573985813
----------------- GUNIV3DAIUSDC2-A -----------------
MSE:  0.004277496065327018
MAE:  0.03143902846083341
KL:  0.872090930099072
TV:  0.015719514230416703
